In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
import numpy as np
import pandas as pd

import os

### Read NSE 500 companies

In [4]:
input_data_folder = '../data/'

In [5]:
nse_500 = pd.read_csv(os.path.join(input_data_folder, 'ind_nifty500list.csv'))
nse_500.head()
nse_500.shape

,Company Name,Industry,Symbol,Series,ISIN Code
0,3M India Ltd.,SERVICES,3MINDIA,EQ,INE470A01017
1,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,EQ,INE012A01025
2,AIA Engineering Ltd.,INDUSTRIAL MANUFACTURING,AIAENG,EQ,INE212H01026
3,APL Apollo Tubes Ltd.,METALS,APLAPOLLO,EQ,INE702C01019
4,AU Small Finance Bank Ltd.,FINANCIAL SERVICES,AUBANK,EQ,INE949L01017


(501, 5)

In [6]:
# num_companies = 10
num_companies = nse_500.Symbol.nunique()

input_companies_symbol = nse_500['Symbol'].values[:num_companies]
input_companies_symbol[:10]
len(input_companies_symbol)

array(['3MINDIA', 'ACC', 'AIAENG', 'APLAPOLLO', 'AUBANK', 'AAVAS',
       'ABBOTINDIA', 'ADANIGAS', 'ADANIGREEN', 'ADANIPORTS'], dtype=object)

501

### Scrap the data

In [8]:
final_basic_stats_list = []
for company_symbol in input_companies_symbol:
#     url = f'https://www.screener.in/company/{company}/consolidated'
    # look at stand-alone data
    url = f'https://www.screener.in/company/{company_symbol}'
    try: 
        
#     print(url)
        response = rq.get(url)
    #     print(response.status_code)
        soup = bs(response.text, "html.parser")  # parse the html page
        basic_features_soup = soup.find_all(class_='row-full-width')
    #     print(len(basic_features_soup))
        basic_features_list = basic_features_soup[0].find_all(class_='four columns')    
        basic_stats = [f.get_text() for f in basic_features_list]

        basic_stats = [f.lower().strip().replace('\n', '').replace('  ', '').replace(' ', '_') for f in basic_stats]
    #     basic_stats

        company_stats_dict = {}
        company_stats_dict['symbol'] = company_symbol
        for f in basic_stats:
            s = f.split(":")
            if len(s)==2:
                company_stats_dict[s[0]] = s[1]
    #     print(basic_stats_dict)
        final_basic_stats_list.append(list(company_stats_dict.values()))
    except IndexError:
        print(f'Having issue with scraping {company_symbol} data. Continuing ...')
        pass

final_basic_stats_list[:5]

Having issue with scraping RNAM data. Continuing ...
Having issue with scraping TATAGLOBAL data. Continuing ...


[['3MINDIA',
  '19,907cr.',
  '17,676',
  '1,808',
  '65.71',
  '0.00%',
  '29.16%',
  '19.02%',
  '9.49%',
  '10.00'],
 ['ACC',
  '22,427cr.',
  '1,194',
  '613.53',
  '16.73',
  '1.17%',
  '18.99%',
  '12.18%',
  '12.52%',
  '10.00'],
 ['AIAENG',
  '15,398cr.',
  '1,631',
  '358.69',
  '28.47',
  '1.66%',
  '17.45%',
  '12.34%',
  '15.96%',
  '2.00'],
 ['APLAPOLLO',
  '3,101cr.',
  '1,247',
  '452.08',
  '22.68',
  '1.12%',
  '16.31%',
  '13.15%',
  '25.78%',
  '10.00'],
 ['AUBANK',
  '12,752cr.',
  '417.50',
  '65.52',
  '18.90',
  '0.18%',
  '8.97%',
  '14.70%',
  '41.16%',
  '10.00']]

In [9]:
company_stats_dict.keys()

dict_keys(['symbol', 'market_cap', 'current_price', 'book_value', 'stock_p/e', 'dividend_yield', 'roce', 'roe', 'sales_growth_(3yrs)', 'face_value'])

In [10]:
company_stats_df = pd.DataFrame(final_basic_stats_list, 
                              columns=company_stats_dict.keys())
company_stats_df.head()
company_stats_df.shape

,symbol,market_cap,current_price,book_value,stock_p/e,dividend_yield,roce,roe,sales_growth_(3yrs),face_value
0,3MINDIA,"19,907cr.","17,676","1,808",65.71,0.00%,29.16%,19.02%,9.49%,10.00
1,ACC,"22,427cr.","1,194",613.53,16.73,1.17%,18.99%,12.18%,12.52%,10.00
2,AIAENG,"15,398cr.","1,631",358.69,28.47,1.66%,17.45%,12.34%,15.96%,2.00
3,APLAPOLLO,"3,101cr.","1,247",452.08,22.68,1.12%,16.31%,13.15%,25.78%,10.00
4,AUBANK,"12,752cr.",417.50,65.52,18.90,0.18%,8.97%,14.70%,41.16%,10.00


(499, 10)

In [18]:
change_col_names = {'stock_p/e': 'stock_pe',
                    'sales_growth_(3yrs)': 'sales_growth_3yrs'
                   }
company_stats_df.rename(change_col_names,axis=1, inplace=True)

In [20]:
company_stats_df.sample(5)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
317,MINDAIND,"8,787cr.",335.10,41.87,62.55,0.33%,15.45%,13.77%,12.65%,2.00
440,TVSMOTOR,"18,705cr.",393.70,70.46,28.09,0.89%,23.67%,21.56%,17.92%,1.00
249,JKPAPER,"2,053cr.",115.20,114.52,4.29,3.04%,24.77%,23.31%,10.14%,10.00
489,VOLTAS,"21,981cr.",664.30,113.13,44.53,0.60%,17.00%,12.32%,9.14%,1.00
288,LTI,"25,741cr.","1,480",271.00,17.05,1.89%,44.24%,33.15%,16.95%,1.00


### Format the data frame

In [21]:
# format values
def format_col_values(x):
    return x.replace('cr.', '').replace(',', '').replace('%', '') 

In [22]:
df = company_stats_df.copy()

In [23]:
df.sample(5)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
441,TAKE,"1,493cr.",100.95,51.54,,0.99%,2.38%,2.10%,-2.33%,1.00
473,MCDOWELL-N,"44,855cr.",617.30,43.10,55.97,0.00%,20.95%,23.18%,2.88%,2.00
344,OFSS,"26,397cr.","3,075",452.05,21.02,0.00%,51.68%,33.37%,0.49%,5.00
58,BERGEPAINT,"42,278cr.",435.35,24.43,86.48,0.44%,29.93%,19.90%,12.65%,1.00
108,COX&KINGS,56.50cr.,3.20,173.18,0.38,31.25%,8.55%,6.00%,76.06%,5.00


In [24]:
cols_to_process = ['market_cap', 'current_price', 'book_value', 'stock_pe',
       'dividend_yield', 'roce', 'roe', 'sales_growth_3yrs', 'face_value']

for col in cols_to_process:
    df[col] = df[col].apply(lambda x: format_col_values(x))

In [25]:
df.sample(5)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
128,LALPATHLAB,11171,1340,111.68,55.08,0.45,33.22,22.15,15.03,10.00
64,BHARTIARTL,175282,341.55,191.66,,0.57,0.01,-4.42,-6.30,5.00
225,INDIACEM,2451,79.10,169.08,20.32,1.01,4.79,1.17,9.99,10.00
446,TATAINVEST,4042,798.80,1546,27.62,2.50,2.01,1.85,-10.31,10.00
473,MCDOWELL-N,44855,617.30,43.10,55.97,0.00,20.95,23.18,2.88,2.00


In [26]:
# replace '', '%', '' values as np.NaN
def format_df(df):
    df = df.replace('cr.', np.NaN).replace('%', np.NaN).replace('', np.NaN)
    return df

In [27]:
df = format_df(df)

In [28]:
df.sample(5)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
150,FORTIS,10422,138.05,110.78,15.92,0.00,5.12,1.98,2.39,10.00
158,GMRINFRA,9899,16.40,19.39,NaN,0.00,1.30,-3.85,-14.74,1.00
247,JKCEMENT,7979,1033,374.39,18.58,0.97,14.35,13.00,6.49,10.00
328,NIITTECH,8642,1385,272.98,31.42,0.00,23.48,18.88,10.44,10.00
474,VGUARD,9663,226.05,21.05,52.64,0.35,25.98,20.07,11.63,1.00


In [29]:
df.isna().sum()

symbol                0
market_cap            0
current_price         0
book_value            1
stock_pe             42
dividend_yield        1
roce                  1
roe                   2
sales_growth_3yrs    12
face_value            0
dtype: int64

In [30]:
df.shape

(501, 10)

### Why values are missing from scraped data

In [31]:
df.sample(5)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
259,JINDALSAW,2545,79.60,199.08,4.63,2.51,11.90,8.22,17.11,2.00
425,SUNDRMFAST,9601,456.95,90.77,22.76,1.12,27.15,25.03,15.46,1.00
497,ZEEL,24156,251.50,74.25,13.32,1.39,35.52,22.69,17.69,1.00
284,LTTS,16676,1603,234.17,22.12,1.31,42.42,31.82,17.65,2.00
437,TTKPRESTIG,8472,6112,832.29,44.38,0.41,26.80,17.63,9.77,10.00


In [37]:
df.isna().sum()

symbol                0
market_cap            0
current_price         0
book_value            1
stock_pe             42
dividend_yield        1
roce                  1
roe                   0
sales_growth_3yrs    12
face_value            0
dtype: int64

#### Companies having missing sales_growth column

In [32]:
missing_sales_growth = df['sales_growth_3yrs'].isna() == True
df[missing_sales_growth]

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
8,ADANIGREEN,10643,68.05,13.80,NaN,0.00,5.72,-2.45,NaN,10.00
28,ARVINDFASN,2488,424.05,230.04,45.94,0.00,5.97,4.78,NaN,4.00
52,BANDHANBNK,57216,479.55,93.89,26.36,0.63,10.48,18.96,NaN,10.00
118,DALBHARAT,15047,779.80,392.83,161.79,0.26,1.37,1.17,NaN,2.00
146,FINEORG,5176,1688,165.84,40.01,0.41,40.95,28.14,NaN,5.00
212,ISEC,8532,264.85,32.11,18.19,3.55,53.06,52.02,NaN,5.00
236,INDOSTAR,1968,213.30,328.38,7.81,0.94,10.13,9.98,NaN,10.00
271,KPITTECH,2614,95.35,33.69,31.47,0.77,18.50,17.99,NaN,10.00
346,ORIENTELEC,3384,159.50,14.45,44.46,0.63,29.12,24.72,NaN,1.00
435,TCIEXP,2657,692.85,69.66,35.38,0.43,44.23,30.83,NaN,2.00


#### Missing roe

In [36]:
missing_roe = df['roe'].isna() == True
df[missing_roe]

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
430,SUZLON,1303,2.45,-13.20,NaN,0.00,-5.41,NaN,-24.64,2.00
447,TATAMTRDVR,2642,51.95,NaN,NaN,NaN,NaN,NaN,NaN,2.00


In [ ]:
# these companies have no roe; we can replace roe with 0
df.loc[missing_roe, 'roe'] = 0

#### Missing roce

In [38]:
missing_roce = df['roce'].isna() == True
df[missing_roce]

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
447,TATAMTRDVR,2642,51.95,NaN,NaN,NaN,NaN,0,NaN,2.00


#### Missing stock_pe

- Companies that are losing money do not have a P/E ratio

In [75]:
missing_stock_pe = df['stock_pe'].isna()
df[missing_stock_pe]

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
8,ADANIGREEN,10643,68.05,13.80,NaN,0.00,5.72,-2.45,NaN,10.00
10,ADANIPOWER,22872,59.30,44.41,NaN,0.00,5.11,-2.40,-42.63,10.00
12,ABCAPITAL,17290,78.50,33.12,NaN,0.00,1.29,0.27,190,10.00
20,ALBK,9900,26.60,24.53,NaN,0.00,0.92,-85.92,-3.70,10.00
25,ANDHRABANK,4924,16.50,44.11,NaN,0.00,4.08,-23.24,2.39,10.00
40,BASF,4268,985.85,326.38,NaN,0.51,1.33,-2.45,8.26,10.00
54,BANKINDIA,19498,59.50,127.20,NaN,0.00,2.98,-15.48,-0.83,10.00
55,MAHABANK,6407,11.00,9.85,NaN,0.00,1.28,-61.01,-5.98,10.00
64,BHARTIARTL,175282,341.55,191.66,NaN,0.57,0.01,-4.42,-6.30,5.00
72,BBTC,7533,1080,36.84,NaN,0.09,2.08,-7.69,0.14,2.00


### Convert to right datatype

In [57]:
df.dtypes

symbol               object
market_cap           object
current_price        object
book_value           object
stock_pe             object
dividend_yield       object
roce                 object
roe                  object
sales_growth_3yrs    object
face_value           object
dtype: object

In [49]:
cols_type = {}
for col in df.columns.values[1:]:
    cols_type[col]='float32'

companies_info = df.astype(cols_type)

In [55]:
companies_info = np.round(companies_info, 2)
companies_info.head(5)
companies_info.dtypes

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
0,3MINDIA,22361.0,19841.000000,1650.000000,72.540001,0.00,29.160000,19.02,9.490000,10.0
1,ACC,26949.0,1435.000000,584.469971,15.620000,0.98,15.860000,15.14,8.130000,10.0
2,AIAENG,15810.0,1677.000000,336.799988,40.259998,0.54,17.450001,12.34,15.960000,2.0
3,APLAPOLLO,3189.0,1315.000000,415.570007,26.590000,1.05,16.309999,13.15,25.780001,10.0
4,AUBANK,19159.0,652.950012,68.150002,38.680000,0.11,8.970000,14.70,41.160000,10.0


symbol                object
market_cap           float32
current_price        float32
book_value           float32
stock_pe             float32
dividend_yield       float32
roce                 float32
roe                  float32
sales_growth_3yrs    float32
face_value           float32
dtype: object

In [60]:
def round_to_two(x):
    return np.round(x, 2)

In [61]:
for col in df.columns.values[1:]:
    companies_info[col] = companies_info[col].apply(lambda x: round_to_two(x))


In [62]:
companies_info.head()

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
0,3MINDIA,22361.0,19841.00,1650.00,72.54,0.00,29.16,19.02,9.49,10.0
1,ACC,26949.0,1435.00,584.47,15.62,0.98,15.86,15.14,8.13,10.0
2,AIAENG,15810.0,1677.00,336.80,40.26,0.54,17.45,12.34,15.96,2.0
3,APLAPOLLO,3189.0,1315.00,415.57,26.59,1.05,16.31,13.15,25.78,10.0
4,AUBANK,19159.0,652.95,68.15,38.68,0.11,8.97,14.70,41.16,10.0


In [65]:
companies_info.isna().sum()

symbol                0
market_cap            0
current_price         0
book_value            1
stock_pe             42
dividend_yield        1
roce                  1
roe                   0
sales_growth_3yrs    12
face_value            0
dtype: int64

### Analyse missing values

In [77]:
companies_info.dtypes

symbol                object
market_cap           float64
current_price        float64
book_value           float64
stock_pe             float64
dividend_yield       float64
roce                 float64
roe                  float64
sales_growth_3yrs    float64
face_value           float64
dtype: object

In [80]:
companies_info.isna().sum()

symbol                0
market_cap            0
current_price         0
book_value            1
stock_pe             42
dividend_yield        1
roce                  1
roe                   0
sales_growth_3yrs    12
face_value            0
dtype: int64

#### Actually bad companies

In [94]:
missing_stock_pe_ind = companies_info.stock_pe.isna()
neg_sales_growth = companies_info['sales_growth_3yrs'] < 0

bad_pe_ind = (missing_stock_pe_ind) & (neg_sales_growth)
companies_info.loc[missing_stock_pe_ind, :].shape
companies_info.loc[missing_stock_pe_ind, :]

(42, 10)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
8,ADANIGREEN,10643.00,68.05,13.80,NaN,0.00,5.72,-2.45,NaN,10.0
10,ADANIPOWER,22872.00,59.30,44.41,NaN,0.00,5.11,-2.40,-42.63,10.0
12,ABCAPITAL,17290.00,78.50,33.12,NaN,0.00,1.29,0.27,190.00,10.0
20,ALBK,9900.00,26.60,24.53,NaN,0.00,0.92,-85.92,-3.70,10.0
25,ANDHRABANK,4924.00,16.50,44.11,NaN,0.00,4.08,-23.24,2.39,10.0
40,BASF,4268.00,985.85,326.38,NaN,0.51,1.33,-2.45,8.26,10.0
54,BANKINDIA,19498.00,59.50,127.20,NaN,0.00,2.98,-15.48,-0.83,10.0
55,MAHABANK,6407.00,11.00,9.85,NaN,0.00,1.28,-61.01,-5.98,10.0
64,BHARTIARTL,175282.00,341.55,191.66,NaN,0.57,0.01,-4.42,-6.30,5.0
72,BBTC,7533.00,1080.00,36.84,NaN,0.09,2.08,-7.69,0.14,2.0


In [95]:
companies_info.loc[bad_pe_ind, :].shape
companies_info.loc[bad_pe_ind, :]

(20, 10)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
10,ADANIPOWER,22872.00,59.30,44.41,NaN,0.00,5.11,-2.40,-42.63,10.0
20,ALBK,9900.00,26.60,24.53,NaN,0.00,0.92,-85.92,-3.70,10.0
54,BANKINDIA,19498.00,59.50,127.20,NaN,0.00,2.98,-15.48,-0.83,10.0
55,MAHABANK,6407.00,11.00,9.85,NaN,0.00,1.28,-61.01,-5.98,10.0
64,BHARTIARTL,175282.00,341.55,191.66,NaN,0.57,0.01,-4.42,-6.30,5.0
90,CENTRALBK,7200.00,17.45,45.89,NaN,0.00,2.40,-30.54,-4.37,10.0
107,CORPBANK,8512.00,14.20,27.63,NaN,0.00,0.97,-46.20,-6.98,2.0
158,GMRINFRA,9899.00,16.40,19.39,NaN,0.00,1.30,-3.85,-14.74,1.0
214,IDBI,23054.00,29.80,48.62,NaN,0.00,-2.05,-51.15,-7.67,10.0
218,IFCI,1153.00,6.80,24.91,NaN,0.00,3.87,-14.20,-17.48,10.0


#### Replace the stock_pe with -1 for these companies

In [96]:
companies_info.loc[bad_pe_ind, 'stock_pe'] = -1

### Handling remaining missing stock_pe

In [98]:
companies_info.isna().sum()

symbol                0
market_cap            0
current_price         0
book_value            1
stock_pe             22
dividend_yield        1
roce                  1
roe                   0
sales_growth_3yrs    12
face_value            0
dtype: int64

In [107]:
missing_stock_pe = companies_info.stock_pe.isna()
row_ind = missing_stock_pe.index[missing_stock_pe]; row_ind
companies_info.loc[missing_stock_pe, :]

Int64Index([  8,  12,  25,  40,  72,  94,  96, 103, 121, 122, 215, 287, 309,
            338, 349, 372, 420, 431, 447, 471, 488, 491],
           dtype='int64')

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
8,ADANIGREEN,10643.00,68.05,13.80,NaN,0.00,5.72,-2.45,NaN,10.0
12,ABCAPITAL,17290.00,78.50,33.12,NaN,0.00,1.29,0.27,190.00,10.0
25,ANDHRABANK,4924.00,16.50,44.11,NaN,0.00,4.08,-23.24,2.39,10.0
40,BASF,4268.00,985.85,326.38,NaN,0.51,1.33,-2.45,8.26,10.0
72,BBTC,7533.00,1080.00,36.84,NaN,0.09,2.08,-7.69,0.14,2.0
94,CHALET,6545.00,319.25,73.10,NaN,0.00,7.99,-0.64,19.23,10.0
96,CHENNPETRO,2459.00,165.10,222.25,NaN,0.00,1.34,-5.20,16.93,10.0
103,COFFEEDAY,927.40,43.90,85.60,NaN,0.00,2.26,-3.34,67.50,10.0
121,DEN,2286.00,47.90,55.18,NaN,0.00,-0.80,-4.27,4.93,10.0
122,DHFL,908.52,28.95,258.17,NaN,0.00,8.17,-11.63,20.88,10.0


In [101]:
companies_info.loc[missing_stock_pe, 'symbol'].values

array(['ADANIGREEN', 'ABCAPITAL', 'ANDHRABANK', 'BASF', 'BBTC', 'CHALET',
       'CHENNPETRO', 'COFFEEDAY', 'DEN', 'DHFL', 'IDFCFIRSTB',
       'LAKSHVILAS', 'MRPL', 'NETWORK18', 'PCJEWELLER', 'PNB', 'SPARC',
       'SWANENERGY', 'TATAMTRDVR', 'UNIONBANK', 'IDEA', 'WELCORP'],
      dtype=object)

In [110]:
companies_info

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
0,3MINDIA,22361.0,19841.00,1650.00,72.54,0.00,29.16,19.02,9.49,10.0
1,ACC,26949.0,1435.00,584.47,15.62,0.98,15.86,15.14,8.13,10.0
2,AIAENG,15810.0,1677.00,336.80,40.26,0.54,17.45,12.34,15.96,2.0
3,APLAPOLLO,3189.0,1315.00,415.57,26.59,1.05,16.31,13.15,25.78,10.0
4,AUBANK,19159.0,652.95,68.15,38.68,0.11,8.97,14.70,41.16,10.0
5,AAVAS,12500.0,1599.00,235.00,65.39,0.00,10.86,11.62,54.94,10.0
6,ABBOTINDIA,22657.0,10662.00,945.22,46.73,0.61,37.36,24.00,12.06,10.0
7,ADANIGAS,14309.0,130.10,10.11,52.99,0.19,24.84,24.06,15.05,1.0
8,ADANIGREEN,10643.0,68.05,13.80,NaN,0.00,5.72,-2.45,NaN,10.0
9,ADANIPORTS,80305.0,395.25,100.86,28.20,0.05,13.41,13.88,4.93,2.0


In [109]:
companies_info.drop(row_ind)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
0,3MINDIA,22361.0,19841.00,1650.00,72.54,0.00,29.16,19.02,9.49,10.0
1,ACC,26949.0,1435.00,584.47,15.62,0.98,15.86,15.14,8.13,10.0
2,AIAENG,15810.0,1677.00,336.80,40.26,0.54,17.45,12.34,15.96,2.0
3,APLAPOLLO,3189.0,1315.00,415.57,26.59,1.05,16.31,13.15,25.78,10.0
4,AUBANK,19159.0,652.95,68.15,38.68,0.11,8.97,14.70,41.16,10.0
5,AAVAS,12500.0,1599.00,235.00,65.39,0.00,10.86,11.62,54.94,10.0
6,ABBOTINDIA,22657.0,10662.00,945.22,46.73,0.61,37.36,24.00,12.06,10.0
7,ADANIGAS,14309.0,130.10,10.11,52.99,0.19,24.84,24.06,15.05,1.0
9,ADANIPORTS,80305.0,395.25,100.86,28.20,0.05,13.41,13.88,4.93,2.0
10,ADANIPOWER,22872.0,59.30,44.41,-1.00,0.00,5.11,-2.40,-42.63,10.0


In [103]:
companies_info.drop(axis=0, index=missing_stock_pe)

KeyError: '[False False False False False False False False  True False False False\n  True False False False False False False False False False False False\n False  True False False False False False False False False False False\n False False False False  True False False False False False False False\n False False False False False False False False False False False False\n False False False False False False False False False False False False\n  True False False False False False False False False False False False\n False False False False False False False False False False  True False\n  True False False False False False False  True False False False False\n False False False False False False False False False False False False\n False  True  True False False False False False False False False False\n False False False False False False False False False False False False\n False False False False False False False False False False False False\n False False False False Fa

In [102]:
missing_stock_pe

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8       True
9      False
10     False
11     False
12      True
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25      True
26     False
27     False
28     False
29     False
       ...  
471     True
472    False
473    False
474    False
475    False
476    False
477    False
478    False
479    False
480    False
481    False
482    False
483    False
484    False
485    False
486    False
487    False
488     True
489    False
490    False
491     True
492    False
493    False
494    False
495    False
496    False
497    False
498    False
499    False
500    False
Name: stock_pe, Length: 501, dtype: bool

### Write to .csv

In [63]:
! pwd

/Users/manishb-imac/personal-projects/clustering-nse-500/notebooks


In [64]:
companies_info.to_csv('../output_data/nse_500_company_info.csv', 
                        index=False)

In [12]:
class TempClass():
    
    @property
    def class_name(cls):
        return cls.__class__.__name__
    
    print(class_name)